In [2]:
import pyspark
import numpy
numpy.version.version
from __future__ import print_function #to treat print as function print(..)

import os
print(os.getcwd())

/home/bitnami/Desktop


In [43]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf

# #Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)

In [10]:
!echo $SHELL
df_out = !df -h
df_out
type(df_out)

/bin/bash


IPython.utils.text.SList

In [11]:
%magic

In [5]:
import pyspark
import numpy
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

# by default reads from hdfs is jps shows its on... hadoop fs -ls
data = sc.textFile("sample_svm_data.txt")
parsedData = data.map(parsePoint)

print(parsedData)
print(data.take(2))
# result = parsedData.collect()
# print(result)

# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

# Save and load model spark version >=1.3
# cannot override output results
model.save(sc, "mllib/pythonLogisticRegressionWithLBFGSModel")
sameModel = LogisticRegressionModel.load(sc,"mllib/pythonLogisticRegressionWithLBFGSModel")




PythonRDD[60] at RDD at PythonRDD.scala:48
[u'1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0', u'0 2.857738033247042 0 0 2.619965104088255 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0']
Training Error = 0.366459627329


In [6]:
my_rdd = sc.parallelize(xrange(100))
print(my_rdd.collect())
print(my_rdd.take(5))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[0, 1, 2, 3, 4]


In [23]:
from pyspark.ml.classification import LogisticRegression

# Load training data
# from pyspark.mllib.util import MLUtils
# training = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt")

val sqlcontext = new org.apache.spark.sql.SQLContext(sc)

training = sqlcontext.read.format("libsvm").load("sample_libsvm_data.txt") #spark.sql



lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

SyntaxError: invalid syntax (<ipython-input-23-9ff18fd80a6e>, line 7)

In [16]:
from pyspark.mllib.util import MLUtils

examples = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt")

print(examples.collect())

[LabeledPoint(0.0, (692,[127,128,129,130,131,154,155,156,157,158,159,181,182,183,184,185,186,187,188,189,207,208,209,210,211,212,213,214,215,216,217,235,236,237,238,239,240,241,242,243,244,245,262,263,264,265,266,267,268,269,270,271,272,273,289,290,291,292,293,294,295,296,297,300,301,302,316,317,318,319,320,321,328,329,330,343,344,345,346,347,348,349,356,357,358,371,372,373,374,384,385,386,399,400,401,412,413,414,426,427,428,429,440,441,442,454,455,456,457,466,467,468,469,470,482,483,484,493,494,495,496,497,510,511,512,520,521,522,523,538,539,540,547,548,549,550,566,567,568,569,570,571,572,573,574,575,576,577,578,594,595,596,597,598,599,600,601,602,603,604,622,623,624,625,626,627,628,629,630,651,652,653,654,655,656,657],[51.0,159.0,253.0,159.0,50.0,48.0,238.0,252.0,252.0,252.0,237.0,54.0,227.0,253.0,252.0,239.0,233.0,252.0,57.0,6.0,10.0,60.0,224.0,252.0,253.0,252.0,202.0,84.0,252.0,253.0,122.0,163.0,252.0,252.0,252.0,253.0,252.0,252.0,96.0,189.0,253.0,167.0,51.0,238.0,253.0,253.0,190.0